<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/AccessDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [ ]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-C3Q8"

In [ ]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

In [ ]:
# @title Configure
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_tag_rid = "2-35RM" # @param {type:"string"}
compare_value = "Cup/Disk_Ratio" #@param {type:"string"}["Diagnosis", "Image_Quality", "Cup/Disk_Ratio"]
initial_diagnosis_tag_rid = "C1T4"
angle_two_rid = '2SK6'


In [ ]:
# @title Example of Graded Test Dataset

Graded_test_initial_diag = EA.image_tall(dataset_rid, initial_diagnosis_tag_rid)
Graded_test_grader_diag = EA.image_tall(dataset_rid, diagnosis_tag_rid)
# Graded_test_gold = EA.image_tall(dataset_rid, "2-5GXP")
long, wide = EA.reshape_table([Graded_test_initial_diag, Graded_test_grader_diag], compare_value)


# long.to_csv("GradedTest_diag_long.csv")
# wide.to_csv("GradedTest_diag_wide.csv")


In [ ]:
long

In [ ]:
wide

In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)